In [1]:
import sys
import os
import math
import random
import heapq 
import time
import copy
import numpy as np
import pandas as pd
from functools import reduce
from scipy.spatial.distance import pdist
from PIL import Image
import matplotlib.pyplot as plt
import cv2
#import faiss
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.set_device(5)
print (torch.cuda.current_device())

5


In [2]:
#1. Read data with List storage Data:[name],I:[img],Y[type]
image_dir = '/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New' #the path of images
trainset = pd.read_csv("/data/fjsdata/ASOCT/Cataract/CBIR_Cataract_ncp_train.csv" , sep=',')#load dataset
#print('The length of test set is %d'%trainset.shape[0])
testset = pd.read_csv("/data/fjsdata/ASOCT/Cataract/CBIR_Cataract_ncp_test.csv" , sep=',')#load testset
#print('The length of test set is %d'%testset.shape[0])

#read train image with CV
trData, trI, trYN, trYC, trYP = [],[],[],[],[]
for index, row in trainset.iterrows():#column: name,id,lr,N,C,P  
    if row['name'].endswith(".jpg"):
        try:
            image_path = os.path.join(image_dir, row['name'])
            img = cv2.resize(cv2.imread(image_path).astype(np.float32), (1024, 1024))
            trData.append(row['name'])
            trI.append(img)
            trYN.append(row['N'])
            trYC.append(row['C'])
            trYP.append(row['P'])
        except:
            print(row['name']+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(trData),trainset.shape[0]))
        sys.stdout.flush()
print('The length of train set is %d'%len(trData))
#read test image with CV
teData, teI, teYN, teYC, teYP = [],[],[],[],[]
for index, row in testset.iterrows():#column: name,id,lr,N,C,P  
    if row['name'].endswith(".jpg"):
        try:
            image_path = os.path.join(image_dir, row['name'])
            img = cv2.resize(cv2.imread(image_path).astype(np.float32), (1024, 1024))
            teData.append(row['name'])
            teI.append(img)
            teYN.append(row['N'])
            teYC.append(row['C'])
            teYP.append(row['P'])
        except:
            print(row['name']+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(teData),testset.shape[0]))
        sys.stdout.flush()
print('The length of train set is %d'%len(teData))

1349 / 29304 C020_20180514_100234_R_CASIA2_LGC_010.jpg:/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/C020_20180514_100234_R_CASIA2_LGC_010.jpg
11175 / 29304 C020_20180514_100234_R_CASIA2_LGC_008.jpg:/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/C020_20180514_100234_R_CASIA2_LGC_008.jpg
11426 / 29304 C020_20180514_100234_R_CASIA2_LGC_002.jpg:/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/C020_20180514_100234_R_CASIA2_LGC_002.jpg
17059 / 29304 C020_20180514_100234_R_CASIA2_LGC_004.jpg:/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/C020_20180514_100234_R_CASIA2_LGC_004.jpg
19051 / 29304 C020_20180514_100234_R_CASIA2_LGC_000.jpg:/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/C020_20180514_100234_R_CASIA2_LGC_000.jpg
19279 / 29304 C020_20180514_100234_R_CASIA2_LGC_016.jpg:/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/C020_20180514_100234_R_CASIA2_LGC_016.jpg
22192 / 29304 c0151_20181119_110748_L_CASIA2_LGC_030.jpg:/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/c0151_20181119_110748_L_CASIA2_LGC_030.jp

In [6]:
#2. define Attention-based Hashing network with pytorch
class HNet(nn.Module): #deep Hashint Network:DHNet
    def __init__(self,inChannels=3):
        super(HNet, self).__init__()
        #(channels, Height, Width)
        #layer1: Convolution, (3,1024,1024)->(16,512,512)
        self.conv1 = nn.Conv2d(in_channels=inChannels, out_channels=16, kernel_size=3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU(inplace=True)
        #layer2: max pooling,(16,512,512)->(16,256,256)
        self.maxpool = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        self.bn2 = nn.BatchNorm2d(16)
        #layer3: Convolution, (16,256,256)->(8,128,128)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, padding=1, stride=2)
        self.bn3 = nn.BatchNorm2d(8)
        self.relu2 = nn.ReLU(inplace=True)
        #layer4: mean pooling, (8,128,128)->(8,64,64)
        self.avgpool1 = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        self.bn4 = nn.BatchNorm2d(8)
        #layer5: Convolution, (8,64,64)->(4,32,32)
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, padding=1, stride=2)
        self.bn5 = nn.BatchNorm2d(4)
        self.relu3 = nn.ReLU(inplace=True)
        #layer6: mean pooling, (4,32,32)->(4,16,16)
        self.avgpool2 = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        self.bn6 = nn.BatchNorm2d(4)
        #layer7: fully connected, 4*16*16->512
        self.fcl1 = nn.Linear(4*16*16,512)
        self.relu4 = nn.ReLU(inplace=True)
        #layer8: Hashing layer, 512->16
        self.fcl2 = nn.Linear(512,16)#
        self.tanh = nn.Tanh() #{-1,1}
              
    def forward(self,x):
        #input: (batch_size, in_channels, Height, Width)
        #output: (batch_size, out_channels, Height, Width)
        #layer1: convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        #layer2: max pooling
        x = self.maxpool(x)
        x = self.bn2(x)
        #layer3: Convolution
        x = self.conv2(x)
        x = self.bn3(x)
        x = self.relu2(x)
        #layer4: mean pooling
        x = self.avgpool1(x)
        x = self.bn4(x)
        #layer5: Convolution
        x = self.conv3(x)
        x = self.bn5(x)
        x = self.relu3(x)
        #layer6: mean pooling
        x = self.avgpool2(x)
        x = self.bn6(x)
        #layer7:fully connected
        x = x.view(x.size(0),-1) #transfer three dims to one dim
        x = self.fcl1(x)
        x = self.relu4(x)
        #layer8: Hashing layer
        x = self.fcl2(x)
        x = self.tanh(x)
                
        return x
    
#https://pytorch-cn.readthedocs.io/zh/latest/    
#https://github.com/filipradenovic/cnnimageretrieval-pytorch/blob/master/cirtorch/layers/functional.py
class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
    
    def forward(self,h1,h2,y): 
        #h1=h2:NxD,y:N
        dim = h1.shape[1]
        euc_dist = F.pairwise_distance(h1, h2, p=2, eps=1e-06) # Calcualte Euclidean Distance
        sim_term = 0.5*(1-y)*euc_dist #penalize the similar iamge pairs when y=0
        unsim_term = 0.5*y*torch.clamp(self.margin*dim-euc_dist,0)#penalize the unsimlar image pairs when y =1
        reg_term = self.alpha * ( torch.sum((torch.abs(h1)-1),dim=1) + torch.sum((torch.abs(h2)-1),dim=1) ) #regularization term
        #loss = torch.mean(sim_term + unsim_term + reg_term) 
        loss = torch.sum(sim_term + unsim_term+ reg_term) 
        return loss

#test network: valid
x1 = torch.rand(10,3,1024,1024)#.cuda()
x2 = torch.rand(10,3,1024,1024)#.cuda()
y = torch.FloatTensor([0,1,1,0,1,0,0,0,1,1])#.cuda()
model = HNet()#.cuda()
criterion  = HashLossFunc(margin=0.5)#.cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
for epoch in range(10):
    optimizer.zero_grad()
    
    out1 = model(x1)#out.grad_fn
    out2 = model(x2)
    loss = criterion(out1,out2,y)
    print (loss.item())
    loss.backward()
    optimizer.step()
    #observe the variant of model.parameters
    for i in model.named_parameters():
        print(i[0])
        print(i[1][0][0][0])
        break
#output
x3 = torch.rand(10,3,1024,1024)#.cuda()
out3 = model(x3)
print (out3)
out3 = torch.sign(out3) #Binarization,[-1,1]->{-1,1}
print (out3)
print (out3.size())

17.03159523010254
conv1.weight
tensor([ 0.1704, -0.1849,  0.1505])
14.555537223815918
conv1.weight
tensor([ 0.1694, -0.1852,  0.1502])
11.206185340881348
conv1.weight
tensor([ 0.1690, -0.1857,  0.1505])
7.917587757110596
conv1.weight
tensor([ 0.1690, -0.1864,  0.1511])
6.534246444702148
conv1.weight
tensor([ 0.1694, -0.1864,  0.1514])
5.623223304748535
conv1.weight
tensor([ 0.1700, -0.1862,  0.1516])
4.265881061553955
conv1.weight
tensor([ 0.1707, -0.1863,  0.1520])
2.8265511989593506
conv1.weight
tensor([ 0.1711, -0.1865,  0.1524])
2.5538811683654785
conv1.weight
tensor([ 0.1713, -0.1869,  0.1529])
2.1924242973327637
conv1.weight
tensor([ 0.1717, -0.1875,  0.1530])
tensor([[ 0.2517, -0.6320,  0.0867, -0.2645,  0.1796,  0.1508,  0.3877,
          0.2545,  0.5201,  0.0066,  0.4533,  0.5320,  0.3209, -0.2749,
         -0.4564, -0.3753],
        [-0.0125,  0.2979, -0.0201,  0.1015, -0.1684, -0.5525, -0.4656,
         -0.3066, -0.4149,  0.2464, -0.3827, -0.3939, -0.3632,  0.0782,
         

In [ ]:
#3.train and evaluate model
def onlineGenImgPairs(batchSize):
    idx_sf = random.sample(range(0, len(trYN)),2*batchSize)
    trI1_sf, trI2_sf, trY1_sf, trY2_sf = [],[],[],[]
    flag = 0
    for i in idx_sf:
        if flag==0:
            trI1_sf.append(trI[i])
            trY1_sf.append(trYN[i])
            flag =1
        else:
            trI2_sf.append(trI[i])
            trY2_sf.append(trYN[i])
            flag =0
    trY_sf = np.where((np.array(trY1_sf)-np.array(trY2_sf))!=0,1,0)
    return np.array(trI1_sf),np.array(trI2_sf),trY_sf
        
#define model
model = HNet().cuda()
criterion  = HashLossFunc(margin=0.5).cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
batchSize = 10
epochSize = len(trYN)//batchSize+1
losses = []
for epoch in range(2*epochSize):#iteration
    #grad vanish
    optimizer.zero_grad() 
    #genenate training images pair
    trI1_sf, trI2_sf, trY_sf = onlineGenImgPairs(batchSize)
    I1_batch = torch.from_numpy(trI1_sf).type(torch.FloatTensor).cuda()
    I2_batch = torch.from_numpy(trI2_sf).type(torch.FloatTensor).cuda()
    Y_batch = torch.from_numpy(trY_sf).type(torch.FloatTensor).cuda()
    #forword
    X1_batch = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
    X2_batch = model(I2_batch.permute(0, 3, 1, 2))
    #binary-like loss
    loss = criterion(X1_batch,X2_batch,Y_batch)
    #backward
    loss.backward()
    #update parameters
    optimizer.step()
    #show loss
    sys.stdout.write('\r {} / {} : loss = {}'.format(epoch, 2*epochSize, float('%0.6f'%loss.item())))
    sys.stdout.flush()     
    losses.append(loss.item())
print("mean_loss = %.6f" % (np.mean(losses)))

#release gpu memory
#model = model.cpu()
#torch.cuda.empty_cache()
#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize+1
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(model(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(teI) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(model(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#train data with list: trData, trI, trF, trY
#test data with list: teData, teI, teF, teY
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = teYN[i]
        map_item_score = {}
        for j, trVal in enumerate(trF):
            map_item_score[j] = pdist(np.vstack([teVal,trVal]),'hamming')
        ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in ranklist:
            dtype = trYN[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 182 / 5860 : loss = 18.463573